# Dataset

CSV file for the bitcoin price for the time period of 01.01.2012 10:01 until 23.02.2025 00:00.

In [ ]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi
from zipfile import ZipFile

ZIP_PATH = "data/zip/"
ZIP_NAME = "bitcoin-historical-data.zip"

RAW_PATH = "data/raw/"
RAW_NAME = "btcusd_1-min_data.csv"

# Validate folders exist
os.makedirs(ZIP_PATH, exist_ok=True)
os.makedirs(RAW_PATH, exist_ok=True)

api = KaggleApi()
api.authenticate()
api.dataset_download_files("mczielinski/bitcoin-historical-data", path=ZIP_PATH)

# Extract zip
zip_file = ZipFile(ZIP_PATH + ZIP_NAME)
zip_file.extractall(path=RAW_PATH)

# Remove zip
os.remove(ZIP_PATH + ZIP_NAME)

In [7]:
# Process dataset

import pandas as pd
import os
from datetime import datetime
import pytz
import shutil

PROCESSED_PATH = "data/processed/"
PROCESSED_NAME = "btcusd_1-min_data_processed.csv"

# Validate folders exist
os.makedirs(PROCESSED_PATH, exist_ok=True)


# Function to convert Unix timestamp to formatted datetime string
def convert_timestamp(ts):
    dt = datetime.fromtimestamp(float(ts), tz=pytz.UTC)
    return dt.strftime("%Y-%m-%d %H:%M:%S%z")


# Copy the original file to processed directory
shutil.copy2(RAW_PATH + RAW_NAME, PROCESSED_PATH + PROCESSED_NAME)

# Convert Timestamp column to datetime format
df = pd.read_csv(PROCESSED_PATH + PROCESSED_NAME)
df["datetime"] = df["Timestamp"].apply(convert_timestamp)

# Save the processed data
df.to_csv(PROCESSED_PATH + PROCESSED_NAME, index=False)

/var/folders/13/dlt4y0gs1cd1f515fyqz0ggm0000gn/T/ipykernel_34753/2885437102.py:26: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(PROCESSED_PATH + PROCESSED_NAME)
